In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import datetime
import matplotlib.pyplot as plt

#Tab1: BMI Tracker
bmi_log = []

def process_data(height, height_unit, weight, weight_unit, date_str):
    if not date_str:
        return "กรุณาป้อนวันที่ (DD-MM-YYYY)"
    try:
        date = datetime.datetime.strptime(date_str, "%d-%m-%Y").date()
    except ValueError:
        return "เกิดข้อผิดพลาด: โปรดป้อนวันที่ในรูปแบบ DD-MM-YYYY ตัวอย่างเช่น 01-02-2024", None, None

    #unit selection
    height_cm = height*2.54 if height_unit=="inches" else height
    weight_kg = weight*0.453592 if weight_unit=="lbs" else weight
    height_m = height_cm/100
    bmi = round(weight_kg/(height_m**2),2)
    #BMI catagory
    if bmi < 18.5:
        category="Underweight"
    elif bmi < 25:
        category="Normal"
    elif bmi < 30:
        category="Overweight"
    else:
        category="Obese"
    date_str_formatted = date.strftime("%d-%m-%Y")
    found = False
    for i, (log_date, log_bmi) in enumerate(bmi_log):
        if log_date == date_str_formatted:
            bmi_log[i] = (date_str_formatted, bmi)
            found = True
            break
    if not found:
        bmi_log.append((date_str_formatted, bmi))
    bmi_log.sort(key=lambda x: datetime.datetime.strptime(x[0], "%d-%m-%Y"))

    summary = (
        f"วันที่: {date_str_formatted}\n"
        f"ส่วนสูง: {round(height_cm,2)} cm ({height} {height_unit})\n"
        f"น้ำหนัก: {round(weight_kg,2)} kg ({weight} {weight_unit})\n"
        f"ค่า BMI: {bmi} → {category}"
    )
    #Visual indicator
    fig1, ax1 = plt.subplots(figsize=(6,1.2))
    ax1.set_xlim(10,40)
    ax1.set_ylim(0,1)
    ax1.axvspan(10,18.5,color="skyblue",alpha=0.5)
    ax1.axvspan(18.5,25,color="lightgreen",alpha=0.5)
    ax1.axvspan(25,30,color="khaki",alpha=0.5)
    ax1.axvspan(30,40,color="salmon",alpha=0.5)
    ax1.axvline(bmi,color="red",linestyle="--",linewidth=2)
    ax1.text(bmi,0.6,f"{bmi}",color="red",ha="center")
    ax1.axis("off")
    #BMI Over Time grapgh
    fig2, ax2 = plt.subplots(figsize=(5,3))
    dates = [d for d,_ in bmi_log]
    values = [v for _,v in bmi_log]
    ax2.plot(dates,values,marker="o")
    ax2.set_title("BMI Over Time")
    ax2.set_xlabel("Date")
    ax2.set_ylabel("BMI")
    ax2.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    return summary, fig1, fig2

tab1 = gr.Interface(
    fn=process_data,
    inputs=[
        gr.Number(label="Height"),
        gr.Radio(["cm","inches"], label="Height Unit", value="cm"),
        gr.Number(label="Weight"),
        gr.Radio(["kg","lbs"], label="Weight Unit", value="kg"),
        gr.Textbox(label="Date (DD-MM-YYYY)") # Changed from gr.Date
    ],
    outputs=[gr.Textbox(label="ผลลัพธ์"), gr.Plot(label="BMI Indicator"), gr.Plot(label="BMI Over Time")],
    title="BMI Tracker"
)

#Tab2: BMR/TDEE
import gradio as gr

def calculate_calorie_needs(age, gender, height, weight, activity_level):
    # คำนวณ BMR
    if gender=="male":
        bmr = 88.362 + 13.397*weight + 4.799*height - 5.677*age
    else:
        bmr = 447.593 + 9.247*weight + 3.098*height - 4.33*age

    # คำนวณ TDEE
    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725,
        "extra active": 1.9
    }
    tdee = bmr * activity_factors.get(activity_level, 1.2)

    # Calories สำหรับเป้าหมายต่าง ๆ
    weight_loss = tdee - 500
    maintenance = tdee
    weight_gain = tdee + 500

    # แสดงผล
    result = f"""
**BMR:** {bmr:.2f} cal/day
**TDEE:** {tdee:.2f} cal/day

**Calories Needed for Goals:**
- Weight Loss: {weight_loss:.2f} cal/day
- Maintenance: {maintenance:.2f} cal/day
- Weight Gain: {weight_gain:.2f} cal/day
"""
    return result

# Gradio Interface
tab2 = gr.Interface(
    fn=calculate_calorie_needs,
    inputs=[
        gr.Number(label="Age (years)"),
        gr.Radio(["male","female"], label="Gender"),
        gr.Number(label="Height (cm)"),
        gr.Number(label="Weight (kg)"),
        gr.Dropdown(["sedentary","lightly active","moderately active","very active","extra active"], label="Activity Level")
    ],
    outputs=gr.Markdown(label="Results"),
    title="BMR/TDEE Calculator with Calorie Goals",
    description="Calculate your Basal Metabolic Rate (BMR), Total Daily Energy Expenditure (TDEE), and daily calories needed for weight loss, maintenance, or weight gain."
)

tab2.launch()

import gradio as gr
import datetime
import matplotlib.pyplot as plt

#Tab3: Food Tracker
state_food_log = gr.State([])

#Log meal function
def log_meal(selected_food, manual_food, calories, meal_time, date_str, tdee, food_log=None):
    if food_log is None:
        food_log = []
    date = datetime.date.today() # Default to today if date_str is empty or invalid
    if date_str:
        try:
            date = datetime.datetime.strptime(date_str, "%d-%m-%Y").date()
        except ValueError:
            food_item = selected_food if selected_food else manual_food
            timestamp = datetime.datetime.combine(datetime.date.today(), datetime.datetime.now().time()).isoformat()
            food_log.append({
                "food_item": food_item,
                "calories": calories,
                "meal_time": meal_time,
                "timestamp": timestamp
            })
            return "เกิดข้อผิดพลาด: โปรดป้อนวันที่ในรูปแบบ DD-MM-YYYY ตัวอย่างเช่น 01-02-2024", food_log


    food_item = selected_food if selected_food else manual_food
    timestamp = datetime.datetime.combine(date, datetime.datetime.now().time()).isoformat()
    food_log.append({
        "food_item": food_item,
        "calories": calories,
        "meal_time": meal_time,
        "timestamp": timestamp
    })
    return f"Logged: {food_item} ({calories} cal) for {meal_time} on {date.strftime('%d-%m-%Y')}", food_log
def display_log_and_total(food_log=None, date_str=None, tdee=0):
    if food_log is None:
        food_log = []
    date = datetime.date.today() # Default to today if date_str is empty or invalid
    if date_str:
        try:
            date = datetime.datetime.strptime(date_str, "%d-%m-%Y").date()
        except ValueError:
            log_str = "เกิดข้อผิดพลาด: โปรดป้อนวันที่ในรูปแบบ DD-MM-YYYY ตัวอย่างเช่น 01-02-2024"
            daily_total = 0
            meal_totals = {"Breakfast":0, "Lunch":0, "Dinner":0}
            for item in food_log:
                entry_date = datetime.datetime.fromisoformat(item["timestamp"]).date()
                if entry_date == datetime.date.today():
                    daily_total += item["calories"]
                    meal_totals[item["meal_time"]] += item["calories"]
            log_str += f"\n**Daily Total Calories (Today):** {daily_total} cal\n"
            for meal, cal in meal_totals.items():
                log_str += f"- {meal}: {cal} cal\n"
            if tdee > 0:
                diff = daily_total - tdee
                status = "At TDEE" if diff==0 else ("Above TDEE" if diff>0 else "Below TDEE")
                log_str += f"\n**TDEE:** {tdee:.2f} cal → Difference: {diff:.2f} cal ({status})"
            return log_str


    daily_total = 0
    meal_totals = {"Breakfast":0, "Lunch":0, "Dinner":0}
    log_str = f"**Food Log for {date.strftime('%d-%m-%Y')}**\n"
    for item in food_log:
        entry_date = datetime.datetime.fromisoformat(item["timestamp"]).date()
        if entry_date == date:
            log_str += f"- {item['meal_time']}: {item['food_item']} ({item['calories']} cal)\n"
            daily_total += item["calories"]
            meal_totals[item["meal_time"]] += item["calories"]
    log_str += f"\n**Daily Total Calories:** {daily_total} cal\n"
    for meal, cal in meal_totals.items():
        log_str += f"- {meal}: {cal} cal\n"

    # Compare to TDEE
    if tdee is not None and tdee > 0: # Added is not None check
        diff = daily_total - tdee
        status = "At TDEE" if diff==0 else ("Above TDEE" if diff>0 else "Below TDEE")
        log_str += f"\n**TDEE:** {tdee:.2f} cal → Difference: {diff:.2f} cal ({status})"
    return log_str


# Plot daily calories by meal for last 7 days
def plot_daily_calories_by_meal(food_log=None):
    if food_log is None:
        food_log = []
    today = datetime.date.today()
    last7 = [today - datetime.timedelta(days=i) for i in range(6,-1,-1)]
    meals = ["Breakfast","Lunch","Dinner"]
    data = {meal:[0]*7 for meal in meals}
    for i, day in enumerate(last7):
        for item in food_log:
            entry_date = datetime.datetime.fromisoformat(item["timestamp"]).date()
            if entry_date == day:
                data[item["meal_time"]][i] += item["calories"]
    fig, ax = plt.subplots(figsize=(10,5))
    bottom = [0]*7
    for meal in meals:
        ax.bar([d.strftime('%d-%m') for d in last7], data[meal], bottom=bottom, label=meal)
        bottom = [bottom[j]+data[meal][j] for j in range(7)]
    ax.set_xlabel("Date")
    ax.set_ylabel("Calories")
    ax.set_title("Daily Calories by Meal (Last 7 Days)")
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig

#Inputs for Tab3
food_dropdown = gr.Dropdown(["Apple","Banana","Chicken Breast","Broccoli","Rice","Pasta"], label="Select Food")
food_manual = gr.Textbox(label="Or Enter Food")
calorie_input = gr.Number(label="Calories")
meal_input = gr.Radio(["Breakfast","Lunch","Dinner"], label="Meal")
date_input = gr.Textbox(label="Date (DD-MM-YYYY, optional)") # Changed to gr.Textbox
tdee_input = gr.Number(label="Enter Your TDEE (calories)")

output_log = gr.Textbox(label="Status", interactive=False)
output_summary = gr.Markdown(label="Food Log and Daily Total")
output_plot = gr.Plot(label="Daily Calories by Meal")
log_button = gr.Button("Log Meal")

#Build Tab3
with gr.Blocks(title="Food Tracker") as tab3:
    gr.Markdown("## Food Tracker")
    gr.Markdown("Log your food items and track daily calories by meal.")

    with gr.Row():
        food_dropdown.render()
        food_manual.render()
        calorie_input.render()
        meal_input.render()
        date_input.render()
        tdee_input.render()
    log_button.render()
    output_log.render()
    output_summary.render()
    output_plot.render()

    state_food_log = gr.State([])

    log_button.click(
        fn=log_meal,
        inputs=[food_dropdown, food_manual, calorie_input, meal_input, date_input, tdee_input, state_food_log],
        outputs=[output_log, state_food_log]
    ).then(
        fn=display_log_and_total,
        inputs=[state_food_log, date_input, tdee_input],
        outputs=output_summary
    ).then(
        fn=plot_daily_calories_by_meal,
        inputs=[state_food_log],
        outputs=output_plot
    )


#Combine Tabs
demo = gr.TabbedInterface([tab1, tab2, tab3], ["BMI Tracker","BMR/TDEE Calculator","Food Tracker"])
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b8ad3143992205321.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c604ae9f0ac27ca4d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
